In [1]:
import numpy as np
import pandas as pd
import re
import os
import requests
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [2]:
# If glove embeds is not in word2vec form then first convert it then load it
if os.path.isfile('pretrained_embeds/gensim_glove_vectors.txt'):
    glove_model = KeyedVectors.load_word2vec_format("pretrained_embeds/gensim_glove_vectors.txt", binary=False)
else:
    glove2word2vec(glove_input_file="pretrained_embeds/glove.twitter.27B.25d.txt", word2vec_output_file="pretrained_embeds/gensim_glove_vectors.txt")
    glove_model = KeyedVectors.load_word2vec_format("pretrained_embeds/gensim_glove_vectors.txt", binary=False)

def get_embed(word):
    # Case folding
    word = word.lower()
    try:
        return (glove_model.get_vector(word))
    except:
        return (glove_model.get_vector('unk'))

In [3]:
get_embed('test')

array([ 1.1768  ,  0.82329 , -0.19366 , -0.25328 ,  0.99367 , -0.1751  ,
        0.95619 , -0.14049 ,  0.90307 ,  0.77942 ,  0.052748,  0.015829,
       -3.0639  ,  0.79883 ,  0.97166 ,  0.1536  ,  0.54858 , -0.062755,
       -1.1394  , -0.53928 , -0.49389 , -0.17549 , -0.41542 ,  0.62815 ,
       -0.33548 ], dtype=float32)

In [4]:
data = pd.read_csv('data.csv')
data.head()

,label_bullying,text_message
0,0,yeah I got 2 backups for all that. I just hate...
1,0,I hate using my BB but love my iPhone. Haven'...
2,1,Get fucking real dude.
3,1,She is as dirty as they come and that crook ...
4,1,why did you fuck it up. I could do it all day...


In [5]:
total = 0
ones = 0

data_embeds = []
label = []

stop_words = set(stopwords.words('english'))

for index, row in data.iterrows():

    sent = row['text_message']
    sent = sent.replace('\n','')                                # Remove new lines
    sent = sent.replace('\t',' ')                               # Remove tabs
    sent = re.sub(r'http\S+', '', sent, flags=re.MULTILINE)     # Remove urls
    sent = re.sub(r'[^\w\s]','', row['text_message'])           # Remove punctuations
    sent = sent.lower().strip()                                 # Case folding
    sent = word_tokenize(sent)                                  # Tokenization
#     sent = [i for i in sent if i not in stop_words]
    
    temp = [get_embed(i) for i in sent]
    temp = np.array(temp)
    temp = temp.mean(axis=0)
    data_embeds.append(temp)
    label.append(row['label_bullying'])

# Train-Test split
partition = int(0.8*len(data))
train = data_embeds[:partition]
train_y = label[:partition]

test = data_embeds[partition:]
test_y = label[partition:]

In [6]:
logreg = LogisticRegression()
logreg.fit(train, train_y)
logreg.score(test, test_y)

/home/yash/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8973922902494331

In [7]:
clf = svm.SVC(C=10, gamma='scale')
clf.fit(train, train_y)
clf.score(test, test_y)

0.9217687074829932